In [ ]:
from Bio import SeqIO
import os
import re
from collections import defaultdict
from datetime import datetime
from datetime import date

#  Pépinière
This notebook contains a bunch of scripts that can process and clean up data from GISAID/BVBRC. It takes any number of fasta files and combines them into one final file. It requires metadata with time, species and continent of isolation

Specify the fasta files and metadata files here and specify which website they were downloaded from. The sequences from the different files will be concatenated into a single file and duplicates will be removed during the process.

GISAID files should be dowloaded with fasta header format: Type|  DNA INSDC   | Isolate name   | DNA Accession no., YYYY-MM-DD format and both space replacement/removal checked.<p>
BVBRC files should be downloaded in standard format and will be reformatted later

In [ ]:
# tuple of files containing the sequences
# (<path>, <GISAID/BVBRC>)
seq_files=(('/Users/threecats/Data/FluSeq/trees/h5_all/raw/gisaid_all-h5-170101.fasta', 'GISAID'),
           ('/Users/threecats/Data/FluSeq/trees/h5_all/raw/gisaid_all-h5-170101-230209.fasta', 'GISAID'),
           ('/Users/threecats/Data/FluSeq/trees/h5_all/raw/BVBRC_all-h5-230214-1.fasta', 'BVBRC'),
           ('/Users/threecats/Data/FluSeq/trees/h5_all/raw/BVBRC_all-h5-230214-2.fasta', 'BVBRC'))

# tuple of files containing metadata
# (<path>, <GISAID/BVBRC>)
metadata_files=(('/Users/threecats/Data/FluSeq/trees/h5_all/raw/gisaid_mdata_all-h5-170101.tsv', 'GISAID'),
                ('/Users/threecats/Data/FluSeq/trees/h5_all/raw/gisaid_mdata_all-h5-170101-230214.tsv', 'GISAID'),
                ('/Users/threecats/Data/FluSeq/trees/h5_all/raw/BVBRC_mdata_all-h5-230214-1.txt', 'BVBRC'),
                ('/Users/threecats/Data/FluSeq/trees/h5_all/raw/BVBRC_mdata_all-h5-230214-2.txt', 'BVBRC'))

# path the species list if used
species_list='/Users/threecats/Data/git/pepiniere/species_list_sorted.txt'

# string pointing at the directory where you want the data
output_dir='/Users/threecats/Data/FluSeq/trees/h5_all'

# name of final dataset
name='allH5'

# 0. Set up
Import packages, set up functions and variables

In [ ]:
# under what length should we look for a second ORF
min_orf_len=1000 # (nts)
# what is the lowest ORF length acceptable
orf_cutoff=0.9 # (relative to average)

# should we output report files?
report_files = True

# should we try to correct common metadata errors?
corr_mdata = True

# should we throw out all files without minimal metadata (ie date)
purge_nomdata = True

# should we only keep on copy of all unique sequences?
purge_unique = True

# should we remove ORFs without stop codon?
purge_nostop = True

# sequences with proportion of undeterminate characters over the threshold will be removed
# set to 1 to not remove sequences with indetermined positions
max_indeterminate=1

# make a file with all short/nostop orfs and the initial sequence?
write_badorf=False

continents=('North_America','South_America','Europe','Oceania','Africa','Asia','Antarctica')

seq_counts={}
for folder in ('temp','data','tree','metadata','annotations'):
    if not os.path.exists(f'{output_dir}/{folder}/'):
        os.mkdir(f'{output_dir}/{folder}/')
if report_files==True:
    if not os.path.exists(f'{output_dir}/reports/'):
        os.mkdir(f'{output_dir}/reports/')

version='23-02-14'


In [ ]:
# this function removes illegal characters from strain names and metadata
# also strips whitespaces from front and back
def remove_illegal_chars(id):
    id=id.strip(' ')
    id = '_'.join(id.split(' '))
    # there probs is a more elegant way to do this, but its fast enough right now
    for char in ('(',')',';',':','.',",","'",'?','（','）',' '):
        id = '-'.join(id.split(char))
    return id

# this function fixes common errors in metadata and returns a formatted string
# mdata should be list date  host    host group    continent   country passage history
# species_dict should be dict species:group
def correct_mdata(strain, mdata):
    report_data=[]
    fixed_date, fixed_continent = '',''
    date_cutoff=2 # decade cut-off for 19XX or 20XX date guessing

    # fixing date
    date_parts = fix_date(mdata[0]).split('-')
    if date_parts[1] != 'XX' and int(date_parts[1]) > 12:
        report_data.append[f'{strain}: fixed date: "{mdata[0]}" -> "{date_parts[0]}-{date_parts[2]}-{date_parts[1]}"']
        fixed_date =f'{date_parts[0]}-{date_parts[2]}-{date_parts[1]}'
    else:
        fixed_date=fix_date(mdata[0])
    if fixed_date=='XXXX-XX-XX':
    # no date provided, or date grivesouly misformatted
    # try to find date in strain name, else just keep as missing and output warning
        strain_date4, strain_date2=strain.rstrip('/').split('/')[-1][:4], strain.strip('/').split('/')[-1][:2]
        if len(strain_date4)==4 and strain_date4.isnumeric():
        # 4 digit number at end of strain name
            fixed_date=strain_date4+'-XX-XX'
        elif strain_date2.isnumeric():
        # no 4 digit, but 2 digit number at end of strain name
            if int(strain_date2[0])<=date_cutoff:
                fixed_date=f'20{strain_date2}-XX-XX'
            else:
                fixed_date=f'19{strain_date2}-XX-XX'

        if fixed_date != 'XXXX-XX-XX':
            report_data.append(f'{strain}: date "{mdata[0]}" could not be parsed, guessed as "{fixed_date}"')
        else:
            print(f'warning, no date for {strain}')
            report_data.append(f'WARNING no date for {strain}')
    # fixing geography
    if remove_illegal_chars(mdata[3]) not in continents and remove_illegal_chars(mdata[4]) in continents:
        fixed_continent =f'{remove_illegal_chars(mdata[4])}\t{remove_illegal_chars(mdata[3])}'
        report_data.append(f'{strain}: fixed continent: "{mdata[3]}/{mdata[4]}" -> "{mdata[4]}/{mdata[3]}"')
    elif remove_illegal_chars(mdata[3]) not in continents and remove_illegal_chars(mdata[4]) not in continents:
        fixed_continent = f'unknown\t{remove_illegal_chars(mdata[4])}' 
        report_data.append(f'{strain}: fixed continent: "{mdata[3]}/{mdata[4]}" -> "unknown/{mdata[4]}"')
    else:
        fixed_continent = f'{remove_illegal_chars(mdata[3])}\t{remove_illegal_chars(mdata[4])}'
    return(f'{fixed_date}\t{mdata[1]}\t{mdata[2]}\t{fixed_continent}\t{mdata[5]}', report_data)

def fix_date(date):
    d=date.split('-')
    if ''.join(d).isnumeric():
    # only numbers, good sign
        if len(date)<10:
        # lower precision date
            if len(d[0])!=4:
            # MM-YYYY format probably??? turn around
                date=f"{d[1]}-{d[0]}"

        elif len(d[2])==4:
        # date in DD-MM-YYYY
            date=f"{d[2]}-{d[1]}-{d[0]}"   
    else:
    # letters in date, uh oh. for now just ditch anything but year
        for part in d:
            if len(part)==4 and part.isnumeric():
            # this one should be the year, else would not have 4 digits
                date=part+'-XX-XX'
                break
        else:
        # can't find a 4 digit year, either missing date or grievously misformatted
            date='XXXX-XX-XX'

    # everything should be numbers or XXXX by now, just pad it all in case it's needed
    while len(date)<10:
        date+='-XX'

    return date

def inderterminate_ratio(seq):
    return(sum([0 if c in ('A','T','C','G') else 1 for c in seq])/len(seq))

# 1. Merging files

### Renaming
Standardizing fasta id format into a GISAID-like \<subtype\>|\<accession number\>|\<strain name\>. For GISAID, sequences without accession number will have the EPI number instead

Also concatenates the sequences into a single temp file.

In [ ]:
# re-initialize counter
seq_counts['total']=0

with open(f'{output_dir}/temp/all.fasta','w') as fout:
    for file in seq_files:
        
        if file[1]=='GISAID':
            for record in SeqIO.parse(file[0], 'fasta'):
                rec_split=record.id.split('|')
                if rec_split[1]!='':
                # has an accession number
                    record_id=f'{rec_split[0][4:]}|{rec_split[1]}|{rec_split[2]}|'
                else:
                # has no accession number  
                    record_id=f'{rec_split[0][4:]}|EPI{rec_split[3]}|{rec_split[2]}|'
                fout.write(f'>{record_id}\n{record.seq}\n')
                seq_counts['total']+=1
       
        elif file[1]=='BVBRC':
            # easiest way to deal with these is actually reading in metadata
            # the fasta is formatted by idiots  >:(
            # dict is accession_number:(subtype, strain name)
            record_ids={}
            for metafile in metadata_files:
                if metafile[1]=='BVBRC':
                    with open(metafile[0], 'r') as f:
                        f.readline()
                        for line in f:
                            l=line.split('\t')
                            record_ids[l[43][1:-1]]=f"{l[21][1:-1]}{'Nx' if len(l[21][1:-1])==2 else ''}|{l[43][1:-1]}|{'_'.join(l[15][1:-1].split(' '))}|**gb"
            for record in SeqIO.parse(file[0], 'fasta'):
                fout.write(f'>{record_ids[record.id[5:]]}\n{record.seq}\n')
                seq_counts['total']+=1
        else:
            print(f'cannot recognize {file[1]} format for {file[0]}, has to be GISAID or BVBRC, skipping')
print(f'combined sequence total: {seq_counts["total"]}')

### First de-duplication round
Here we only remove sequences that are 100% identical and with the same accession number/strain name. This is done in 2 rounds, first looks for duplicate accession numbers, second for duplicate strain names. I am assuming that all sequences with the same accession number are identical, I don't actually check for it. For strain names it gets more complicated since the sequences might differ.

BVBRC agrees with Genbank while GISAID seems to make mistakes, so the former is prioritised. Additionally if a Genbank sequence is updated, GISAID does not update the entry while BVBRC does (I think? IRD used to).

In [ ]:
# re-initialize counter
seq_counts['dedup']=0
blacklist=[]
db_counts={'gisaid':0,'both':0,'bvbrc':0}
report_data={}

id_dict=defaultdict(list)
with open(f'{output_dir}/temp/all.fasta','r') as f:
    # get accession number from fasta id, ignore fasta without id
    for line in f:
        if line.startswith('>'):
            an=line.split('|')[1]
            if an != '':
                id_dict[an].append(line.strip().split('|',1)[1])

for records in id_dict.values():
    if len(records)>1:
        if ''.join(records).count('**gb')==1:
        # only one seq from BVBRC, keep that one
            k=''
            v=[]
            db_counts['both']+=1
            for record in records:
                if '**gb' not in record:
                    blacklist.append(record)
                    v.append(record)
                else:
                    k=record
            report_data[k]=v

        elif ''.join(records).count('**gb')==0:
        # only GISAID seqs, keep first
            blacklist+=records[1:]
            report_data[records[0]]=records[1:]
        
        else:
        # several BVBRC, keep first
            if ''.join(records).count('**gb')!=len(records):
            # at least one GISAID
                db_counts['both']+=1
            for record in records:
                if '**gb' in record:
                    records.remove(record)
                    report_data[record]=records
                    break
            blacklist+=records

# weed out first batch
seqs_to_keep=defaultdict(list)
with open(f'{output_dir}/temp/id_filtered.fasta', 'w') as f:
    for record in SeqIO.parse(f'{output_dir}/temp/all.fasta', 'fasta'):
        if record.id.split('|',1)[1] in blacklist:
        # found a blacklisted record, remove one entry in blacklist
        # this is important if the same seq is present several times
        # if we don't remove we keep none of them
            blacklist.remove(record.id.split('|',1)[1])
        else:
            if '**gb' not in record.id:
                db_counts['gisaid']+=1
            else:
                db_counts['bvbrc']+=1
            seq_counts['dedup']+=1
            f.write(f'>{record.id}\n{record.seq}\n')

if report_files == True:
# put out file with dupe info
    with open(f'{output_dir}/reports/deduplication.txt', 'w') as f:
        f.write('kept\treplaces\n')
        for k,v in report_data.items():
            f.write(f'{k}\t{",".join(v)}\n')

print(f'found and removed {seq_counts["total"]-seq_counts["dedup"]} duplicate sequences ({(seq_counts["total"]-seq_counts["dedup"])/seq_counts["total"]*100:.2f}%)')
print(f"final set contains {db_counts['gisaid']+db_counts['bvbrc']} sequences,"+
      f" of which {db_counts['gisaid']} from GISAID ({db_counts['gisaid']/(db_counts['gisaid']+db_counts['bvbrc'])*100:.2f}%)"+
      f" and {db_counts['bvbrc']} from BVBRC ({db_counts['bvbrc']/(db_counts['bvbrc']+db_counts['gisaid'])*100:.2f}%),"+
      f" {db_counts['both']} of which were also in GISAID ({db_counts['both']/db_counts['bvbrc']*100:.2f}%)")
if report_files==True:
    print(f'wrote IDs of duplicates to {output_dir}/reports/deduplication.txt')


# 2. Fixing and formatting metadata

GISAID metadata is downloaded as xlsx file and has been converted to tsv. Sadly some people put characters into the metadata that break the whole process so first we have to fix it.<p>
BVBRC metadata just gets concatenated and the quotation marks removed

In [ ]:
with open(f'{output_dir}/metadata/GISAID_fixed.tsv', 'w') as fout:
    # write header
    fout.write('Isolate_Id\tPB2 Segment_Id\tPB1 Segment_Id\tPA Segment_Id\tHA Segment_Id\tNP Segment_Id\tNA Segment_Id\tMP Segment_Id\tNS Segment_Id\tHE Segment_Id\tP3 Segment_Id\tIsolate_Name\tSubtype\tLineage\tPassage_History\tLocation\tHost\tIsolate_Submitter\tSubmitting_Lab\tSubmitting_Sample_Id\tAuthors\tPublication\tOriginating_Lab\tOriginating_Sample_Id\tCollection_Date\tNote\tUpdate_Date\tSubmission_Date\tAntigen_Character\tAnimal_Vaccin_Product\tAdamantanes_Resistance_geno\tOseltamivir_Resistance_geno\tZanamivir_Resistance_geno\tPeramivir_Resistance_geno\tOther_Resistance_geno\tAdamantanes_Resistance_pheno\tOseltamivir_Resistance_pheno\tZanamivir_Resistance_pheno\tPeramivir_Resistance_pheno\tOther_Resistance_pheno\tHost_Age\tHost_Age_Unit\tHost_Gender\tPatient_Status\tZip_Code\tOutbreak\tPathogen_Test_Info\tIs_Vaccinated\tHuman_Specimen_Source\tAnimal_Specimen_Source\tAnimal_Health_Status\tDomestic_Status\tPMID\tPB2 INSDC_Upload\tPB1 INSDC_Upload\tPA INSDC_Upload\tHA INSDC_Upload\tNP INSDC_Upload\tNA INSDC_Upload\tMP INSDC_Upload\tNS INSDC_Upload\tHE INSDC_Upload\tP3 INSDC_Upload')
    done={'',} # keep track of samples already added
    for file in metadata_files:
        if file[1]=='GISAID':
            with open(file[0], 'r') as fin:
                curr_line=''
                fin.readline() # skip header
                for line in fin:
                    if line.startswith('EPI'):
                        if curr_line not in done:
                            fout.write('\n'+curr_line)
                            done.update(curr_line)
                        curr_line=''
                        curr_line+=line.strip()
                    else:
                        curr_line+=line.strip()
                # add last line
                if curr_line not in done:
                    fout.write('\n'+curr_line)
                if file == metadata_files[-1]:
                    fout.write('\n')

with open(f'{output_dir}/metadata/BVBRC_fixed.tsv', 'w') as fout:
    # write header
    fout.write('Genome ID\tGenome Name\tOther Names\tNCBI Taxon ID\tTaxon Lineage IDs\tTaxon Lineage Names\tSuperkingdom\tKingdom\tPhylum\tClass\tOrder\tFamily\tGenus\tSpecies\tGenome Status\tStrain\tSerovar\tBiovar\tPathovar\tMLST\tSegment\tSubtype\tH_type\tN_type\tH1 Clade Global\tH1 Clade US\tH5 Clade\tpH1N1-like\tLineage\tClade\tSubclade\tOther Typing\tCulture Collection\tType Strain\tReference\tGenome Quality\tCompletion Date\tPublication\tAuthors\tBioProject Accession\tBioSample Accession\tAssembly Accession\tSRA Accession\tGenBank Accessions\tSequencing Center\tSequencing Status\tSequencing Platform\tSequencing Depth\tAssembly Method\tChromosome\tPlasmids\tContigs\tSize\tGC Content\tContig L50\tContig N50\tTRNA\tRRNA\tMat Peptide\tCDS\tCoarse Consistency\tFine Consistency\tCheckM Contamination\tCheckM Completeness\tGenome Quality Flags\tIsolation Source\tIsolation Comments\tCollection Date\tCollection Year\tSeason\tIsolation Country\tGeographic Group\tGeographic Location\tOther Environmental\tHost Name\tHost Common Name\tHost Gender\tHost Age\tHost Health\tHost Group\tLab Host\tPassage\tOther Clinical\tAdditional Metadata\tComments\tDate Inserted\tDate Modified\n')
    done={'',} # keep track of samples already added
    for file in metadata_files:
        if file[1]=='BVBRC':
            with open(file[0], 'r') as fin:
                fin.readline() # skip header
                for line in fin:
                    if line not in done:
                        fout.write('\t'.join([l.strip('"') for l in line.strip().split('\t')])+'\n')
                        done.update(line)

### Unify and filter metdata
There's lots of fields we don't really need in the metadata and GISAID and BVBRC use completely different formats. To avoid duplicate entries, we'll also be going through the deduplicated sequence set and ensure one entry/sequence.

We also take this opportunity to correct common (and easily detectable) metadata errors such as American-formatted dates and inverted countries/continents

In [ ]:
# metadata will be read into dicts:
# strain_name:date  host    host group   continent   country passage history

report_data=[]
mdata_counts={'total':0,
              'no date':0,
              'no data':0,
              'not in species':0}
# read in species list
# expanding list of species sorted into groups
missing_species=[]
with open(species_list, 'r') as f:
    species_dict={line.strip().split('\t')[1]:line.split('\t')[0] for line in f}

# read in all GISAID data
md_gisaid={}
with open(f'{output_dir}/metadata/GISAID_fixed.tsv', 'r') as f:
    f.readline() # skip header
    for line in f:
        l=line.split('\t')

        # figure out species group
        species=remove_illegal_chars(l[16]).lower()
        if species in species_dict.keys():
            species_group = species_dict[species]
        else:
            species_group='unknown'
            missing_species.append(species)
     
        md_gisaid[remove_illegal_chars(l[11])]=[l[24], l[16], species_group, l[15].split(' / ')[0].strip('"'), l[15].split(' / ')[1].strip('"') if len(l[15].split(' / '))>1 else '', l[14]]

# read in all BVBRC data
md_bvbrc={}
with open(f'{output_dir}/metadata/BVBRC_fixed.tsv', 'r') as f:
    f.readline() # skip header
    for line in f:
        l=line.split('\t')

        # figure out species group
        species=remove_illegal_chars(l[74]).lower().split('-_gender')[0]
        if species in species_dict.keys():
            species_group = species_dict[species]
        else:
            species_group='unknown'
            missing_species.append(species)

        md_bvbrc[remove_illegal_chars(l[15])]=[l[67], l[74], species_group, l[71].strip('"'), l[70].strip('"'), l[81]]
        
# go through sequences, grab mdata, CORRECT, and put to file
with open(f'{output_dir}/metadata/mdata.tsv', 'w') as f:
    f.write('name\tdate\thost\thost group\tcontinent\tcountry\tpassage history\n')
    for record in SeqIO.parse(f'{output_dir}/temp/id_filtered.fasta', 'fasta'):
        mdata_counts['total']+=1
        strain=remove_illegal_chars(record.id.split('|')[2])
        if '**gb' in record.id:
        # use BCBRV metadata
            if strain not in md_bvbrc.keys():
                print(f'no BVBRC entry for {record.id}')
                report_data.append(f'{strain}: no metadata found in {output_dir}/metadata/BVBRC_fixed.tsv')
                mdata='\t\t\t\t\t'
                mdata_counts['no data']+=1
            else:
                if corr_mdata==True:
                    mdata, report_datum=correct_mdata(strain,md_bvbrc[strain])
                    report_data+=report_datum
                    if mdata.split('\t')[0]=='XXXX-XX-XX':
                        mdata_counts['no date']+=1
                    if mdata.split('\t')[2]=='unknown':
                        mdata_counts['not in species']+=1
                else:
                    mdata='\t'.join(md_bvbrc[strain])
        else:
        # use GISAID metadata
            if strain not in md_gisaid.keys():
                print(f'no GISAID entry for {record.id}')
                report_data.append(f'{strain}: no metadate found in {output_dir}/metadata/GISAID_fixed.tsv')
                mdata='\t\t\t\t\t'
                mdata_counts['no data']+=1
            else:
                if corr_mdata==True:
                    mdata, report_datum=correct_mdata(strain,md_gisaid[strain])
                    report_data+=report_datum
                    if mdata.split('\t')[0]=='XXXX-XX-XX':
                        mdata_counts['no date']+=1
                    if mdata.split('\t')[2]=='unknown':
                        mdata_counts['not in species']+=1
                else:
                    mdata='\t'.join(md_gisaid[strain])
        f.write(f'{remove_illegal_chars(record.id)}\t{mdata}\n')

if missing_species!=[] and report_data==True:
    with open(f'{output_dir}/reports/missing_species.tsv', 'r') as f:
        for species in set(missing_species):
            f.write(f'\t{species}\n')
            print(f'no species list entry for {species}')



if report_data != [] and report_files==True:
    with open(f'{output_dir}/reports/metadata_correction.txt', 'w') as f:
        for report in report_data:
            f.write(report+'\n')

# 3. Final filtering of sequences and ORF finding
Here we do a bunch of things at once:
- Find ORFs for all sequences and filter out sequences without stop codon or with short ORFs
- Filter illegal characters from sequence names
- Filter out sequences with no medata or no date information (optional)
- Filter out 100% identical ORFs and only keep the earliest occurence (optional)
- Filter out ORFs with too many inderterminate positions (optional)

In [ ]:

# initializing counters/lists
i = 0
orfgood = [] # these are ORFs with a start and a stop
orfbad = [] # these are ORFS with a start and a stop but smaller than 90% of the average ORF or
            # or ORFs without stop codon

orf_counters={'nostop':0,
              'short':0,
              'no_mdata':0,
              'too_many_ind':0,
              'dupes':0,
              'kept':0}

# read in sequences
in_seqs=SeqIO.index(f'{output_dir}/temp/id_filtered.fasta', "fasta")

# read in metadata
mdata={}
with open(f'{output_dir}/metadata/mdata.tsv', 'r') as f:
    f.readline() # skip header
    for line in f:
        l=line.strip().split('\t')
        mdata[l[0]]=l[1:]

for record in SeqIO.parse(f'{output_dir}/temp/id_filtered.fasta', 'fasta'): # iterating through sequences
    if purge_nomdata == True and remove_illegal_chars(record.id) not in mdata.keys():
    # no metadata
        orfbad.append((record.id, 'no_mdata', record.seq))
        orf_counters['no_mdata']+=1
    elif purge_nomdata == True and mdata[remove_illegal_chars(record.id)][0]== 'XXXX-XX-XX':
    # no date
        orfbad.append((record.id, 'no_date', record.seq))
        orf_counters['no_mdata']+=1
    else:
    # everything ok, proceed to find ORFs
        seq = record.seq[record.seq.upper().find("ATG"):] # trim 5' end by finding first ATG and slicing 
        
        while len(seq) % 3 != 0: # making seq length a multiple of 3 so biopython doesn't yell at me (Ns will get cut off in next step)
            seq += "N"
        
        seq = seq[:len(seq.translate(to_stop=True))*3+3] # trim 3' end by slicing everything after the stop codon

        # sometimes the first ATG is not the right one, this part also tests the second ATG
        # if the orf from the first was too short
        if len(seq) < min_orf_len: 
            seq2 = record.seq[record.seq[record.seq.upper().find("ATG")+3:].upper().find('ATG')+record.seq.upper().find("ATG")+3:]
            while len(seq2) % 3 !=0 :
                seq2 += "N"
            seq2 = seq2[:len(seq2.translate(to_stop=True))*3+3]
            # if this ORF is longer, replace the previous small one by this one
            if len(seq2) > len(seq):
                seq = seq2

        if purge_nostop==True and seq.translate()[-1:] != "*": # check for stop codon at the end of ORF
            orfbad.append((record.id,'nostop', seq))
            orf_counters['nostop']+=1
        else: # either we're not checking or there is a stop codon
            orfgood.append((record.id,seq))

## sort out short ORFs (90% cut-off)
# 90% is arbitrary, adjust as desired
orf_cutoff_nts = sum([len(orf[1]) for orf in orfgood])/len(orfgood)*orf_cutoff

# turn orfgood inside out to group ids from identical sequences:
orfids=defaultdict(list)
for orf in orfgood:
    orfids[orf[1].upper()].append(orf[0])

replaced=[]
with open(f'{output_dir}/{name}_unique_orfs.fasta', "w") as f:
    for seq in orfids.keys(): # iterate through unique sequences
        if inderterminate_ratio(seq.upper())<max_indeterminate:
        # check indeterminate character ratio
            # check for length, if bad set aside
            if len(seq) < orf_cutoff_nts:
            # check for length, if bad set aside
                for seqid in orfids[seq]:
                    orfbad.append((seqid, 'short', seq))
                    orf_counters['short']+=1
            else:
            # length is ok
                if purge_unique == True:
                    if len(orfids[seq])>1:
                    # not unique sequence, figure out which to keep
                        v=[remove_illegal_chars(seqid) for seqid in orfids[seq]]
                        # find the sequence with earliest date
                        # for incomplete dates, first day of first month is taken
                        earliest_id =v[[datetime.fromisoformat('-01'.join(mdata[seqid][0].split('-XX'))) for seqid in v].index(min([datetime.fromisoformat('-01'.join(mdata[seqid][0].split('-XX'))) for seqid in v]))]
                        v.remove(earliest_id)
                        f.write(f">{earliest_id}\n{seq}\n")
                        replaced.append(f"{earliest_id}\t{','.join(v) if len(v)>1 else v[0]}\n")
                        orf_counters['kept']+=1
                        orf_counters['dupes']+=len(v)
                    else:
                        f.write(f'>{orfids[seq][0]}\n{seq}\n')
                        orf_counters['kept']+=1

                else:
                # we don't care about uniques
                    for seqid in orfids[seq]:
                        f.write(f">{seqid}\n{seq}\n")
        else:
        # too many indeterminate characters
            orfbad.append((orf[0], f'too_many_ind-{inderterminate_ratio(orf[1].upper())}', orf[1]))
            orf_counters['too_many_ind']+=1


# reporting on data
print(f'looked for orfs in {sum(orf_counters.values())} sequences:')
print(f"\t{sum(orf_counters.values())-orf_counters['kept']:>6} ({(sum(orf_counters.values())-orf_counters['kept'])/sum(orf_counters.values())*100:5.2f}%) sequences discarded")
if purge_nomdata==True:
    print(f"\t\t{orf_counters['no_mdata']:>6} ({orf_counters['no_mdata']/sum(orf_counters.values())*100:5.2f}%) had no associated metadata or date")
print(f"\t\t{orf_counters['short']:>6} ({orf_counters['short']/sum(orf_counters.values())*100:5.2f}%) were shorter than {orf_cutoff*100:.0f}% of the average ({orf_cutoff_nts:.0f})")
if purge_nostop==True:
    print(f"\t\t{orf_counters['nostop']:>6} ({orf_counters['nostop']/sum(orf_counters.values())*100:5.2f}%) had no stop codon")
if max_indeterminate!=1:
    print(f"\t\t{orf_counters['too_many_ind']:>6} ({orf_counters['too_many_ind']/sum(orf_counters.values())*100:5.2f}%) had over {max_indeterminate*100:.0f}% indeterminate posiions")
if purge_unique==True:
    print(f"\t\t{orf_counters['dupes']:>6} ({orf_counters['dupes']/sum(orf_counters.values())*100:5.2f}%) were not unique")
print(f"\t{orf_counters['kept']:>6} ({(orf_counters['kept'])/sum(orf_counters.values())*100:5.2f}%) sequences kept and written to {output_dir}/{name}_unique_orfs.fasta")

if report_files==True:
    # sift through all the stuff we exluded
    orfbad_output=[]
    with open(f'{output_dir}/reports/bad_orfs.tsv', 'w') as f:
        for orf in orfbad:
            if orf[1] in ('nostop', 'short'):
                orfbad_output.append(orf)
                detail=len(orf[2])
            elif orf[1]=='too_many_ind':
                detail=f'{inderterminate_ratio(orf[2]):.2f}'
            else:
                detail=''
            f.write(f'{orf[0]}\t{orf[1]}\t{detail}\n')
    
    if orfbad_output!=[] and write_badorf==True:
        with open(f'{output_dir}/temp/orfs_bad.fasta', "w") as f2:
            for item in orfbad_output:
                f2.write(f'>{item[0]}_{item[1]}\n{item[2]}\n')
                f2.write(f'>{item[0]}\n{str(in_seqs[item[0]].seq)}\n')

    # report on replacements
    with open(f'{output_dir}/reports/replaced.tsv', 'w') as f:
        for item in replaced:
            f.write(item)



The dataset should now be ready for alignment tree building.

Of course, there's always more that can be done, starting with getting some stats on the whole process.

# 4. Some info and stats

Run this to output a file that sums up what was done to the data

In [ ]:
with open(f'{output_dir}/parameters.txt', 'w') as f:
    f.write(f'dataset {name} was cleaned up using pépinière v{version} on {date.today()}\n')
    if report_files==True:
        f.write(f"reports were generated and written to {output_dir}/reports\n")
    else:
        f.write('no reports were generated\n')


    f.write(f'\n{"_"*60}\n')
    f.write('INPUT SEQUENCES\n')
    f.write(f'input files were: {seq_files[0][0]} ({seq_files[0][1]} format)\n')
    for seq_file in seq_files[1:]:
        f.write(f"{' '*18}{seq_file[0]} ({seq_file[1]} format)\n")
    f.write(f'\nfound and removed {seq_counts["total"]-seq_counts["dedup"]} duplicate sequences based on accession/EPI numbers ({(seq_counts["total"]-seq_counts["dedup"])/seq_counts["total"]*100:.2f}%)\n')
    f.write(f"final set contains {db_counts['gisaid']+db_counts['bvbrc']} sequences,"+
        f" of which {db_counts['gisaid']} from GISAID ({db_counts['gisaid']/(db_counts['gisaid']+db_counts['bvbrc'])*100:.2f}%)"+
        f" and {db_counts['bvbrc']} from BVBRC ({db_counts['bvbrc']/(db_counts['bvbrc']+db_counts['gisaid'])*100:.2f}%),"+
        f" {db_counts['both']} of which were also in GISAID ({db_counts['both']/db_counts['bvbrc']*100:.2f}%)\n")
    

    f.write(f'\n{"_"*60}\n')
    f.write('INPUT METADATA\n')
    f.write(f'\nmetadata files were: {metadata_files[0][0]} ({metadata_files[0][1]} format)\n')
    for metadata_file in metadata_files[1:]:
        f.write(f"{' '*21}{metadata_file[0]} ({metadata_file[1]} format)\n")
    f.write(f"\nmetadata was{' not' if corr_mdata==False else ''} corrected and consolidated to {output_dir}/metadata/mdata.tsv\n")
    f.write(f"species list used was {species_list} containing {len(species_dict)} entries but missing {len(set(missing_species))} entries\n")
    f.write(f"\nof {mdata_counts['total']} processed metadata entries:\n")
    f.write(f"\t{mdata_counts['no data']:>5} had no metadata info{' '*6}({mdata_counts['no data']/mdata_counts['total']*100:5.2f}%)\n")
    f.write(f"\t{mdata_counts['no date']:>5} had no date info{' '*10}({mdata_counts['no date']/mdata_counts['total']*100:5.2f}%)\n")
    f.write(f"\t{mdata_counts['not in species']:>5} had no species group info ({mdata_counts['not in species']/mdata_counts['total']*100:5.2f}%)\n")



    f.write(f'\n{"_"*60}\n')
    f.write('ORF TRIMMING\n')
    f.write(f'looked for orfs in {sum(orf_counters.values())} sequences:\n')
    f.write(f"\t{sum(orf_counters.values())-orf_counters['kept']:>6} ({(sum(orf_counters.values())-orf_counters['kept'])/sum(orf_counters.values())*100:5.2f}%) sequences discarded\n")
    f.write(f"\t\t{orf_counters['short']:>6} ({orf_counters['short']/sum(orf_counters.values())*100:5.2f}%) were shorter than {orf_cutoff*100:.0f}% of the average ({orf_cutoff_nts:.0f})\n")
    if purge_nomdata==True:
        f.write(f"\t\t{orf_counters['no_mdata']:>6} ({orf_counters['no_mdata']/sum(orf_counters.values())*100:5.2f}%) had no associated metadata or date\n")
    else:
        f.write('\t\tsequences without metadata were not removed from the dataset\n')
    if purge_nostop==True:
        f.write(f"\t\t{orf_counters['nostop']:>6} ({orf_counters['nostop']/sum(orf_counters.values())*100:5.2f}%) had no stop codon\n")
    else:
        f.write('\t\tORFs without stop codon were not removed from the dataset\n')
    if max_indeterminate!=1:
        f.write(f"\t\t{orf_counters['too_many_ind']:>6} ({orf_counters['too_many_ind']/sum(orf_counters.values())*100:5.2f}%) had over {max_indeterminate*100:.0f}% indeterminate posiions\n")
    else:
        f.write('\t\tORFs with indeterminate positions were not removed from the dataset\n')
    if purge_unique==True:
        f.write(f"\t\t{orf_counters['dupes']:>6} ({orf_counters['dupes']/sum(orf_counters.values())*100:5.2f}%) were not unique\n")
    else:
        f.write('\t\tduplicate ORFs were not removed from the dataset\n')
    f.write(f"\t{orf_counters['kept']:>6} ({(orf_counters['kept'])/sum(orf_counters.values())*100:5.2f}%) sequences kept and written to {output_dir}/{name}_unique_orfs.fasta\n")
    f.write(f'\nfinal dataset contains {orf_counters["kept"]} of {seq_counts["total"]} sequences ({orf_counters["kept"]/seq_counts["total"]*100:.2f}%)')


    

# 5. Treework